## Deliverable 2. Create a Customer Travel Destinations Map.

In [15]:
# Dependencies
%matplotlib inline

import matplotlib.pyplot as plt
import json
import requests
import os
import pandas as pd
import numpy as np
import gmaps
import requests

from scipy.stats import linregress
from dotenv import load_dotenv
from citipy import citipy
from datetime import datetime
import time
load_dotenv()
api_key = os.getenv("OpenWeather_API_KEY")
g_key = os.getenv("g_key")

# Configure gmaps
gmaps.configure(api_key=g_key)

In [16]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,Aguas Vermelhas,-15.7472,-41.4600,56.28,98,21,2.39,BR,2022-07-20 05:34:22,few clouds
1,1,Adrar,20.5022,-10.0711,85.60,17,15,5.39,MR,2022-07-20 05:34:23,few clouds
2,2,Albany,42.6001,-73.9662,72.54,87,51,4.88,US,2022-07-20 05:32:08,broken clouds
3,3,Mataura,-46.1927,168.8643,43.48,84,97,2.39,NZ,2022-07-20 05:34:23,overcast clouds
4,4,Tiksi,71.6872,128.8694,34.14,95,100,13.06,RU,2022-07-20 05:34:24,overcast clouds


In [19]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 76


In [20]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
55,55,Alofi,-19.0595,-169.9187,75.09,78,10,16.11,NU,2022-07-20 05:35:41,clear sky
114,114,Valencia,39.3333,-0.8333,75.02,54,0,0.67,ES,2022-07-20 05:37:04,clear sky
182,182,Agirish,61.9247,63.0231,75.54,37,100,8.84,RU,2022-07-20 05:38:29,overcast clouds
203,203,Miandrivazo,-19.5167,45.4667,75.97,39,0,9.66,MG,2022-07-20 05:39:37,clear sky
222,222,Saint-Paul,-21.0096,55.2707,75.31,53,4,5.39,RE,2022-07-20 05:39:45,clear sky
270,270,Ciudad Bolivar,8.1222,-63.5497,75.00,93,100,2.64,VE,2022-07-20 05:41:04,overcast clouds
293,293,Myitkyina,25.3833,97.4000,75.88,100,100,1.39,MM,2022-07-20 05:42:13,overcast clouds
295,295,Ingham,42.6000,-84.3833,75.38,82,40,8.05,US,2022-07-20 05:42:14,scattered clouds
326,326,Grafton,42.2070,-71.6856,75.45,83,0,9.22,US,2022-07-20 05:43:26,clear sky
350,350,Mao,39.8885,4.2658,75.76,94,0,2.30,ES,2022-07-20 05:43:35,fog


In [21]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isna().sum().sum()

len(preferred_cities_df)

28

In [22]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [23]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

length=len(hotel_df)

In [24]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

record_count = 1 

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # Adding ouput counter
    
    print(f"Processing Record {record_count} of {length}", end='\r')
    record_count += 1
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found in {row[0]} ... skipping.")
 
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Hotel not found in Agirish ... skipping.
Hotel not found in Poya ... skipping.
Hotel not found in Sabla ... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [25]:
#TESTERRRR

# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

record_count = 1 

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # Adding ouput counter
    
    print(row[0])
   

Alofi
Valencia
Agirish
Miandrivazo
Saint-Paul
Ciudad Bolivar
Myitkyina
Ingham
Grafton
Mao
Palmer
Poya
Medea
Tiarei
Akhtanizovskaya
Aleksandrov Gay
Monrovia
Cayenne
Rincon
Roma
Mana
Escarcega
Port-Gentil
Kropotkin
Acarau
Sabla
Kutum
Saint Anthony


In [26]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df["Hotel Name"] != ""]
clean_hotel_df.set_index("City")

,Country,Max Temp,Description,Lat,Lng,Hotel Name
City,,,,,,
Alofi,NU,75.09,clear sky,-19.0595,-169.9187,Taloa Heights
Valencia,ES,75.02,clear sky,39.3333,-0.8333,Masia El Pino
Miandrivazo,MG,75.97,clear sky,-19.5167,45.4667,Hôtel Arc En Ciel Miandrivazo
Saint-Paul,RE,75.31,clear sky,-21.0096,55.2707,Boucan Canot
Ciudad Bolivar,VE,75.00,overcast clouds,8.1222,-63.5497,Hotel Edi
Myitkyina,MM,75.88,overcast clouds,25.3833,97.4000,Shwe Phyu Hotel
Ingham,US,75.38,scattered clouds,42.6000,-84.3833,Sunset Mobile Home Park
Grafton,US,75.45,clear sky,42.2070,-71.6856,The Grafton Inn
Mao,ES,75.76,fog,39.8885,4.2658,ARTIEM Capri


In [27]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"


# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [28]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Description</dt><dd>{Description}</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))